In [1]:
import random
from itertools import combinations, product
from functools import lru_cache

# Constants
NUM_GOODS = 2
NUM_THETA = 15

# Generate goods
goods = [f"g{i}" for i in range(NUM_GOODS)]

# Generate powerset (all subsets)
def powerset(lst):
    return [set(comb) for i in range(len(lst)+1) for comb in combinations(lst, i)]

all_subsets = powerset(goods)

# preference_1 = {frozenset(g): random.randint(1, 5) for g in goods}
# preferences_2 =

# # Initial endowments (half of the goods each)
even_split = (NUM_GOODS//2)
print(even_split)

initial_endowment_1 = frozenset(goods[:even_split])  # Agent 1 gets g0, g1, g2
initial_endowment_2 = frozenset(goods[even_split:])  # Agent 2 gets g3, g4, g5
initial_endowment = (initial_endowment_1, initial_endowment_2)
# # Generate Agent 1 preference (theta_1)
# theta_1 = {frozenset(s): random.randint(1, 5) for s in all_subsets}

# # Generate Agent 2 preference types (theta_2_list)
# theta_2_list = []
# for _ in range(NUM_THETA):
#     profile = {frozenset(s): random.randint(1, 5) for s in all_subsets}
#     theta_2_list.append(profile)


1


In [2]:
print(f"INITIAL GOOD ENDOWMENTS: Agent 1 has{initial_endowment_1}, Agent 2 has {initial_endowment_2}")

INITIAL GOOD ENDOWMENTS: Agent 1 hasfrozenset({'g0'}), Agent 2 has frozenset({'g1'})


# EXPERIMENT (GIOVANNI)
The thetas as previously defined made the algorithm run in the correct way but had absurd results, possibly assigning a higher valuation to an empty set than to the full set (something that is against the rules).
I defined separate subsets `preference_1` for agent 1 and `preferences_2` for agent 2, and assign values to the subsets directly (one for every theta in the case of agent 2).

In [3]:
# Define preference_1 based on individual goods (Agent 1's base preferences)
preference_1 = {frozenset({g}): random.randint(1, 5) for g in goods}

# Generate Agent 1 preference (theta_1) based on summing preference_1
theta_1 = {}
for subset in all_subsets:
    subset_utility = sum(preference_1.get(frozenset({good}), 0) for good in subset)
    theta_1[frozenset(subset)] = subset_utility

# Generate Agent 2 preference types (theta_2_list) with varying individual preferences
theta_2_list = []
for _ in range(NUM_THETA):
    # Generate a new preference for individual goods for this Agent 2 type
    preference_2 = {frozenset({g}): random.randint(1, 5) for g in goods}

    # Construct the full preference profile (theta) for this Agent 2 type
    profile = {}
    for subset in all_subsets:
        # Calculate subset utility based on summing the current Agent 2's individual good preferences
        subset_utility = sum(preference_2.get(frozenset({good}), 0) for good in subset)
        profile[frozenset(subset)] = subset_utility
    theta_2_list.append(profile)

# You can add print statements here to check the generated theta_1 and theta_2_list
print("theta_1:", theta_1)
print("theta_2_list (first profile):", theta_2_list[0])
print("theta_2_list (second profile):", theta_2_list[1]) # To see they are different


theta_1: {frozenset(): 0, frozenset({'g0'}): 4, frozenset({'g1'}): 3, frozenset({'g0', 'g1'}): 7}
theta_2_list (first profile): {frozenset(): 0, frozenset({'g0'}): 3, frozenset({'g1'}): 5, frozenset({'g0', 'g1'}): 8}
theta_2_list (second profile): {frozenset(): 0, frozenset({'g0'}): 4, frozenset({'g1'}): 4, frozenset({'g0', 'g1'}): 8}


In [4]:
# Generate all possible outcomes (2^{NUM_GOODS} in total)
outcomes = []
for bits in product([0, 1], repeat=NUM_GOODS):
    agent1_goods = {goods[i] for i in range(NUM_GOODS) if bits[i] == 0}
    agent2_goods = {goods[i] for i in range(NUM_GOODS) if bits[i] == 1}
    outcomes.append((frozenset(agent1_goods), frozenset(agent2_goods)))

assert len(outcomes) == 2 ** NUM_GOODS

# Utility function for Agent 1 with endowment subtraction
def g(outcome):
    agent1_bundle, _ = outcome
    return theta_1.get(agent1_bundle, 0) - theta_1.get(initial_endowment_1, 0)

# Sort outcomes by Agent 1 utility (descending)
outcomes.sort(key=g, reverse=True)

# Utility function for Agent 2 (with endowment subtraction)
def u(theta, outcome):
    _, agent2_bundle = outcome
    return theta.get(agent2_bundle, 0) - theta.get(initial_endowment_2, 0)


In [5]:
@lru_cache(maxsize=None)
def v_cached(X_tuple, Y_tuple):
    X = set(X_tuple)
    Y = set(Y_tuple)
    remaining_outcomes = [o for o in outcomes if o not in Y]
    total = 0
    for theta in theta_2_list:
        best_value = -float("inf")
        for o in remaining_outcomes:
            if all(u(theta, o) >= u(theta, x) for x in X):
                best_value = max(best_value, g(o))
        if best_value != -float("inf"):
            total += best_value
    return total / len(theta_2_list)


In [6]:
CB = None
L = -float("inf")

def SEARCH1(X, Y, w, d):
    global CB, L

    if d > len(outcomes):
        CB = X.copy()
        L = w
        return

    # if d % 10 == 0:
        # print(f"Depth {d}, current best L = {L}")

    od = outcomes[d - 1]
    X_with_od = X | {od}
    v1 = v_cached(tuple(X_with_od), tuple(Y))
    if v1 > L:
        SEARCH1(X_with_od, Y, v1, d + 1)

    new_Y = Y | {od}
    remaining_outcomes = [o for o in outcomes if o not in new_Y]
    condition1 = all(any(u(theta, o) >= 0 for o in remaining_outcomes) for theta in theta_2_list)
    v2 = v_cached(tuple(X), tuple(new_Y))

    if condition1 and v2 > L:
        SEARCH1(X, new_Y, v2, d + 1)

def BnB_DFS():
    global CB, L
    CB = set()
    L = -float("inf")
    SEARCH1(set(), set(), 0, 1)
    return CB


In [7]:
def MCB(theta, CB):
    valid_outcomes = []
    for o in CB:
        if all(u(theta, o) >= u(theta, x) for x in CB):
            valid_outcomes.append(o)
    if not valid_outcomes:
        return None
    return max(valid_outcomes, key=g)


In [8]:
CB_result = BnB_DFS()
print("CB_result:", CB_result)

# # Apply MCB for first agent 2 type
# chosen_outcome = MCB(theta_2_list[0], CB_result)
# print("Chosen outcome for θ[0]:")
# print("  Agent 1 gets:", set(chosen_outcome[0]))
# print("  Agent 2 gets:", set(chosen_outcome[1]))
# print("  Agent 1 utility (g):", g(chosen_outcome))
# print("  Agent 2 utility (u):", u(theta_2_list[0], chosen_outcome))
# print(f"Agent 1 had {initial_endowment_1} with {theta_1.get(initial_endowment_1)} initially, while Agent 2 had {initial_endowment_2} with {theta_2_list[0].get(initial_endowment_2)}")

CB_result: {(frozenset({'g0'}), frozenset({'g1'})), (frozenset({'g0', 'g1'}), frozenset())}


# experiment: verify truthfulness and individual rationality

Helpers to verify that the chosen mechanisms are truthful

In [9]:
from itertools import product

def generate_misreports(theta, goods, value_range=(1, 5)):
    misreports = []
    # Get the base preferences for individual goods from the current theta
    base_preferences = {good: theta.get(frozenset({good}), 0) for good in goods}

    # Generate all possible combinations of values for each good's individual preference
    value_combinations = product(range(value_range[0], value_range[1] + 1), repeat=len(goods))

    for values in value_combinations:
        # Create a new misreported preference for individual goods
        misreported_preference = {frozenset({goods[i]}): values[i] for i in range(len(goods))}

        # Construct the full misreported preference profile (theta)
        misreported_theta = {}
        for subset in all_subsets:
            subset_utility = sum(misreported_preference.get(frozenset({good}), 0) for good in subset)
            misreported_theta[frozenset(subset)] = subset_utility

        misreports.append(misreported_theta)
    return misreports

def verify_truthfulness(true_theta, chosen_outcome, goods, CB_result):
    """
    Verifies that misreporting preferences does not yield a better outcome for Agent 2.

    Args:
        true_theta: The true preference profile of Agent 2.
        chosen_outcome: The outcome chosen by the mechanism for the true_theta.
        goods: The list of goods.
        CB_result: The set of Candidate Best outcomes found by the mechanism.
    """
    true_utility = u(true_theta, chosen_outcome)
    misreports = generate_misreports(true_theta, goods)

    print(f"Verifying truthfulness for a true theta with utility {true_utility} for the chosen outcome.")
    print(f"Generated {len(misreports)} possible misreports.")

    for i, misreported_theta in enumerate(misreports):
        misreported_chosen_outcome = MCB(misreported_theta, CB_result)

        if misreported_chosen_outcome:
            misreported_utility = u(true_theta, misreported_chosen_outcome)
            assert misreported_utility <= true_utility, f"Truthfulness violated! Misreport {i} with outcome {misreported_chosen_outcome} yielded utility {misreported_utility} which is better than true utility {true_utility} with chosen outcome {chosen_outcome}."
        else:
            # If no valid outcome is found for a misreport, it doesn't violate truthfulness
            pass # Or print a message: print(f"Misreport {i}: No valid outcome found in CB_result.")

    print("Truthfulness verified for this theta: All misreports resulted in utility less than or equal to the true utility.")

# Example usage:
# Choose a theta to test (e.g., the first one from theta_2_list)
theta_to_test = theta_2_list[0]

# Find the chosen outcome for this theta using the MCB function
chosen_outcome_for_test_theta = MCB(theta_to_test, CB_result)

# Verify truthfulness for this theta and its chosen outcome
if chosen_outcome_for_test_theta:
    verify_truthfulness(theta_to_test, chosen_outcome_for_test_theta, goods, CB_result)
else:
    print("Could not find a chosen outcome for the selected theta, cannot verify truthfulness.")

Verifying truthfulness for a true theta with utility 0 for the chosen outcome.
Generated 25 possible misreports.
Truthfulness verified for this theta: All misreports resulted in utility less than or equal to the true utility.


In [10]:
# Apply MCB for all Agent 2 types
for i, theta in enumerate(theta_2_list):
    chosen_outcome = MCB(theta, CB_result)

    print(f"\nChosen outcome for θ[{i}]:")
    if chosen_outcome:
        print("  Agent 1 gets:", set(chosen_outcome[0]))
        print("  Agent 2 gets:", set(chosen_outcome[1]))
        print("  Agent 1 utility (g):", g(chosen_outcome))
        print("  Agent 2 utility (u):", u(theta, chosen_outcome))
        # print(f"  Agent 1 had {initial_endowment_1} with utility {theta_1.get(initial_endowment_1)} initially, while Agent 2 had {initial_endowment_2} with {theta.get(initial_endowment_2)}")
        print(f"  Agent 1 had {initial_endowment_1} with utility {g(initial_endowment)} initially (actual initial goods valuation: {theta_1.get(initial_endowment[0])}),  \n Agent 2 had {initial_endowment_2} with utility {u(theta, initial_endowment)} (actual initial goods valuation: {theta.get(initial_endowment[1])})")
        verify_truthfulness(theta, chosen_outcome, goods, CB_result)
    else:
        print("  No valid outcome found in CB_result for this theta.")


Chosen outcome for θ[0]:
  Agent 1 gets: {'g0'}
  Agent 2 gets: {'g1'}
  Agent 1 utility (g): 0
  Agent 2 utility (u): 0
  Agent 1 had frozenset({'g0'}) with utility 0 initially (actual initial goods valuation: 4),  
 Agent 2 had frozenset({'g1'}) with utility 0 (actual initial goods valuation: 5)
Verifying truthfulness for a true theta with utility 0 for the chosen outcome.
Generated 25 possible misreports.
Truthfulness verified for this theta: All misreports resulted in utility less than or equal to the true utility.

Chosen outcome for θ[1]:
  Agent 1 gets: {'g0'}
  Agent 2 gets: {'g1'}
  Agent 1 utility (g): 0
  Agent 2 utility (u): 0
  Agent 1 had frozenset({'g0'}) with utility 0 initially (actual initial goods valuation: 4),  
 Agent 2 had frozenset({'g1'}) with utility 0 (actual initial goods valuation: 4)
Verifying truthfulness for a true theta with utility 0 for the chosen outcome.
Generated 25 possible misreports.
Truthfulness verified for this theta: All misreports resulted